#CYBERBULLYING TWEET IDENTIFICATION MACHINE LEARNING

By : Muhammad Fu'ad Saifuddin

Email : muh.fuad.saifuddin@gmail.com

Linkedin : https://www.linkedin.com/in/muhammad-fuad-saifuddin/

Github : https://github.com/fuad-saifuddin




In [1]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [2]:
tf.__version__

'2.9.2'

In [3]:
import pandas as pd

url='https://drive.google.com/file/d/1g_7mCQKNLt07UwIep1uHviPXb_A6UYxz/view?usp=share_link'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url)

In [4]:
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [6]:
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [7]:
import numpy as np
df['is_cyberbullying'] = np.where(df['cyberbullying_type']=='not_cyberbullying',0,1)
df

,tweet_text,cyberbullying_type,is_cyberbullying
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0
...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity,1
47688,Turner did not withhold his disappointment. Tu...,ethnicity,1
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity,1
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity,1


In [8]:
from sklearn.model_selection import train_test_split

X = df['tweet_text']
y = df['is_cyberbullying']

X_train,X_val,y_train,y_val = train_test_split (X,y,test_size=0.2)

In [9]:
y_train.value_counts()

1    31809
0     6344
Name: is_cyberbullying, dtype: int64

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=8000, oov_token='x')

tokenizer.fit_on_texts(X_train) 
 
seq_train = tokenizer.texts_to_sequences(X_train)
seq_val = tokenizer.texts_to_sequences(X_val)
 
pad_train = pad_sequences(seq_train) 
pad_val = pad_sequences(seq_val)

In [11]:
print(len(tokenizer.index_word))

56190


In [12]:
from imblearn.over_sampling import RandomOverSampler


pad_train_ovsample,y_train_ovsample = RandomOverSampler(random_state=42).fit_resample(pad_train,y_train)

In [13]:
y_train_ovsample.value_counts()

0    31809
1    31809
Name: is_cyberbullying, dtype: int64

In [14]:
len(pad_train_ovsample)

63618

In [15]:
len(pad_train[2])

874

In [16]:
import tensorflow as tf
from keras.layers import Dense, Concatenate, Add
model1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=8000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=8000, output_dim=16),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

merge = Concatenate()([model1.output, model2.output])
merge = Dense(1, activation='sigmoid')(merge)

final_model = tf.keras.models.Model([model1.input,model2.input], merge)

final_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 embedding_input (InputLayer)   [(None, None)]       0           []                               
                                                                                                  
 embedding_1_input (InputLayer)  [(None, None)]      0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 16)     128000      ['embedding_input[0][0]']        
                                                                                                  
 embedding_1 (Embedding)        (None, None, 16)     128000      ['embedding_1_input[0][0]']      
                                                                                              

In [18]:
history = final_model.fit([pad_train_ovsample,pad_train_ovsample], y_train_ovsample, epochs=6, validation_data=([pad_val,pad_val], y_val), verbose=1)

Epoch 1/6
1989/1989 [==============================] - 874s 438ms/step - loss: 0.6048 - accuracy: 0.7001 - val_loss: 0.4941 - val_accuracy: 0.8392
Epoch 2/6
1989/1989 [==============================] - 864s 434ms/step - loss: 0.4419 - accuracy: 0.8438 - val_loss: 0.3962 - val_accuracy: 0.8515
Epoch 3/6
1989/1989 [==============================] - 868s 436ms/step - loss: 0.3739 - accuracy: 0.8659 - val_loss: 0.3733 - val_accuracy: 0.8496
Epoch 4/6
1989/1989 [==============================] - 865s 435ms/step - loss: 0.3365 - accuracy: 0.8786 - val_loss: 0.3655 - val_accuracy: 0.8572
Epoch 5/6
1989/1989 [==============================] - 867s 436ms/step - loss: 0.3110 - accuracy: 0.8883 - val_loss: 0.3852 - val_accuracy: 0.8442
Epoch 6/6
1989/1989 [==============================] - 881s 443ms/step - loss: 0.2937 - accuracy: 0.8946 - val_loss: 0.3829 - val_accuracy: 0.8522
